# **Herència**

**Exemple**

Suposem que tenim una classe `CompteCorrent` per gestionar els comptes corrents dels clients d’una entitat bancària. Aquesta classe guarda la informació del titular del compte, el saldo actual i una llista amb tots els moviments que s’han fet al compte. Per guardar els moviments s’utilitza una classe `Moviment` que guarda una descripció del moviment, l’import del moviment (positiu o negatiu) i la data en què s’ha fet. 
A nivell d’interfície pública, la classe `Moviment` només té els getters i setters dels atributs. La classe `CompteCorrent` té com a interfície pública getters per recuperar el titular i el saldo actual i un mètode per registrar les dades d’un moviment. 


In [1]:
from dataclasses import dataclass, field, InitVar
from typing import List
from datetime import date, datetime

@dataclass
class Moviment:
    _descripcio: str
    _valor: float
    _data: date

    def __init__(self, descripcio: str = "", valor: float = 0.0, data: str = '') -> None:
        self._descripcio = descripcio
        self._valor = valor
        if data == '':
            self._data = date.today()
        else:
            self._data = datetime.strptime(data, '%d/%m/%Y')
        
    @property
    def descripcio(self) -> str:
        return self._descripcio
    
    @descripcio.setter
    def descripcio(self, valor: str) -> None:
        self._descripcio = valor
        
    @property
    def valor(self) -> float:
        return self._valor
    
    @valor.setter
    def valor(self, valor: float) -> None:
        self._valor = valor
        
    @property
    def data(self) -> str:
        return datetime.strftime(self._data, '%d/%m/%Y')
    
    @data.setter
    def data(self, valor: str) -> None:
        self._data = datetime.strptime(valor, '%d/%m/%Y')


In [2]:
@dataclass
class CompteCorrent:
    _titular: str
    _saldo: float = 0.0
    _moviments: List[Moviment] = field(init=False, default_factory=list)
    
    @property
    def titular(self) -> str:
        return self._titular
    
    @property
    def saldo(self) -> float:
        return self._saldo
    
    def afegeix_moviment(self, descripcio: str, valor: float, data: str) -> None:
        moviment = Moviment(descripcio, valor, data)
        self._moviments.append(moviment)
        self._saldo += valor

In [34]:
c = CompteCorrent("ernest", 1000)
print(c.titular, c.saldo)

ernest 1000


In [38]:
c.afegeix_moviment("desc1", 200, "1/1/2022")
c.afegeix_moviment("desc2", -100, "1/1/2022")
print(c.saldo)

1100


**Exemple**

Imaginem ara que l’entitat bancària vol oferir un nou tipus de compte, el Compte Jove, que té una operativa molt similar al compte corrent bàsic, amb la diferència que el Compte Jove permet acumular punts que després es poden bescanviar per regals. Els punts s’acumulen cada cop que el client fa un ingrés en el compte: per cada 100 euros ingressats s’acumula 1 punt. 


#### **Herència**
- És la utilització d’una classe genèrica ja existent (classe base `CompteCorrent`) per crear altres  classes més específiques (classe derivada `CompteJove`) que especialitzen la classe genèrica i extenen/modifiquen la seva funcionalitat.
- Les classes derivades (o també subclasses) hereten els atributs (dades)  i els mètodes (comportament) de la classe base (o també superclasse). 


In [3]:
@dataclass
class CompteJove(CompteCorrent):
    pass
    

In [4]:
cj = CompteJove('ernest')
print(cj.titular, cj.saldo)
cj.afegeix_moviment("desc1", 200, "1/1/2022")
cj.afegeix_moviment("desc2", -100, "1/1/2022")
print(cj.saldo)

ernest 0.0
100.0


- Les classes derivades poden afegir atributs i mètodes específics propis només de la subclasse.
- Les subclasses també poden modificar el comportament dels mètodes de la classe base per adaptar-lo a la seva especifitat

**Exemple**

Podem crear una nova classe `CompteJove` com a classe derivada de `CompteCorrent`:
- Afegim un atribut per guardar els punts acumulats i una propietat per poder recuperar el valor d'aquest atribut. 
- Modifiquem el comportament del mètode `afegeix_moviment` perquè a més a més d'afegir el moviment a la llista i actualitzar el saldo, també s'actualitzi el valor dels punts acumulats. 

In [36]:
@dataclass
class CompteJove(CompteCorrent):
    _punts: int = 0
    
    @property
    def punts(self) -> int:
        return self._punts
       
    def afegeix_moviment(self, descripcio: str, valor: float, data: str) -> None:
        moviment = Moviment(descripcio, valor, data)
        self._moviments.append(moviment)
        self._saldo += valor        
        if valor > 0:
            self._punts += int(valor/100)


In [37]:
cj = CompteJove('ernest', 1000, 100)
print(cj.titular, cj.saldo, cj.punts)
cj.afegeix_moviment("desc1", 200, "1/1/2022")
cj.afegeix_moviment("desc2", -100, "1/1/2022")
print(cj.saldo, cj.punts)

ernest 1000 100
1100 102


### ***Overriding***

És la redefinició d'un mètode de la classe base a la classe derivada per modificar el comportament del mètode a la classe derivada.

Des de la classe derivada podem cridar a la versió del mètode de la classe base i així ens evitem duplicar el codi que sigui comú a la classe base i a la classe derivada. La crida a la versió de la classe base es fa: `super().nom_metode(parametres)`

In [24]:
@dataclass
class CompteJove(CompteCorrent):
    _punts: int = field(init=False,default=0)
    
    @property
    def punts(self) -> int:
        return self._punts
       
    def afegeix_moviment(self, descripcio: str, valor: float, data: str) -> None:
        super().afegeix_moviment(descripcio, valor, data)
        if valor > 0:
            self._punts += int(valor/100)


In [25]:
cj = CompteJove('ernest')
print(cj.titular, cj.saldo, cj.punts)
cj.afegeix_moviment("desc1", 200, "1/1/2022")
cj.afegeix_moviment("desc2", -100, "1/1/2022")
print(cj.saldo, cj.punts)

ernest 1000.0 100
1100.0 102


***Overriding i el mètode __init__***

Quan cridem a un mètode redefinit de la classe derivada, només s'executa el mètode de la classe derivada. No es fa la crida al mètode de la classe base si no fem la crida explícitament des de la classe derivada amb `super().nom_metode(parametres)`

Això és especialment rellevant quan inicialitzem els atributs de les classes amb els mètode `__init__` o `__post_init__`. Veiem-ho amb l'exemple següent:

**Exemple**

Volem crear una classe derivada de la classe `Moviment` per poder gestionar els rebuts com un tipus particular de moviments. En els rebuts, apart de guardar la informació genèrica de qualsevol moviment hi volem guardar el nom de l'entitat que emet el rebut. 

Provem el següent codi per implementar una classe `Rebut` com a classe derivada de `Moviment`

In [8]:
@dataclass
class Rebut(Moviment):
    _entitat: str = ''
    
    def __init__(self, entitat=''):
        self._entitat = entitat
       
    @property
    def entitat(self) -> str:
        return self._entitat
    
    @entitat.setter
    def entitat(self, valor: str) -> None:
        self._entitat = valor

In [9]:
r = Rebut()
print(r.descripcio, r.valor, r.data, r.entitat)

 0.0 04/03/2022 


Si volem que els atributs de la classe base `Moviment` quedin correctament definits i inicialitzats hem de cridar explícitament al mètode `__init__` de la classe des de la classe derivada `Rebut`. 

In [11]:
@dataclass
class Rebut(Moviment):
    _entitat: str = ''
    
    def __init__(self, descripcio: str = "", valor: float = 0.0, data: str = '', entitat=''):
        super().__init__(descripcio, valor, data)
        self._entitat = entitat
       
    @property
    def entitat(self) -> str:
        return self._entitat
    
    @entitat.setter
    def entitat(self, valor: str) -> None:
        self._entitat = valor

In [12]:
r = Rebut('desc', 100, '10/03/2022', 'ent')
print(r.descripcio, r.valor, r.data, r.entitat)

desc 100 10/03/2022 ent
